In [13]:
%load_ext autoreload
%autoreload 2

from Utils import Utils
from rdkit.Chem import AllChem
import torch
embed_dim = 32
smiles = 'OCc1ccccc1CN'
mol = AllChem.MolFromSmiles(smiles)
mol, conf = Utils._generate_conformer(mol)

atom_bond_graph = Utils._get__angle_graph(mol, conf)
bond_angle_graph = Utils._get_bond_angle_graph(mol, conf)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
bond_angle_graph.edata['bond_angle']

tensor([1.9222, 1.9222, 2.0719, 2.1291, 2.0719, 2.1106, 2.0814, 2.1106, 2.0927,
        2.0927, 2.0932, 2.0932, 2.1135, 2.1135, 2.0692, 2.0749, 2.0692, 1.9505,
        2.1390, 1.9505, 2.1291, 2.0814, 2.0749, 2.1390], device='cuda:0')

In [51]:
atom_indexes = (2,0,1,2)
shared_atom_index = [x for x in atom_indexes if atom_indexes.count(x) > 1][0]
ends_indexes = [x for x in atom_indexes if atom_indexes.count(x) == 1]

rdmt.GetAngleRad(conf, ends_indexes[0], shared_atom_index, ends_indexes[1])

0.5800554715383166

In [26]:
*(1,2,3)

{1, 2}

In [5]:
%load_ext autoreload
%autoreload 2

from Utils import Utils
embed_dim = 32
smiles = 'OCc1ccccc1CN'
graph = Utils.smiles_to_graph(smiles)


from FeaturesEmbedding import FeaturesEmbedding
node_embed_layer = FeaturesEmbedding(Utils.FEATURES['atom_feats'], embed_dim).cuda()
edge_embed_layer = FeaturesEmbedding(Utils.FEATURES['bond_feats'], embed_dim).cuda()
node_feats = node_embed_layer.forward(graph.ndata)
edge_feats = edge_embed_layer.forward(graph.edata)

# from SqrtGraphNorm import SqrtGraphNorm
# nn = SqrtGraphNorm().cuda()
# nn.forward(graph, node_embed_layer.forward(g.ndata))

# from SimpleGIN import SimpleGIN
# nn = SimpleGIN(embed_dim).cuda()
# nn.forward(
#     graph,
#     node_embed_layer.forward(graph.ndata),
#     edge_embed_layer.forward(graph.edata)
# )


# from FeaturesEmbedding import FeaturesRBF
# nn = FeaturesRBF(Utils.RBF_PARAMS['bond'], embed_dim).cuda()
# nn.forward(graph.edata).shape


# from FixedRBF import FixedRBF
# centers, gamma = Utils.RBF_PARAMS['bond']['bond_length']
# nn = FixedRBF(centers[:13], gamma).cuda()
# nn.forward(graph.edata['bond_length']).shape


from GeoGNN import GeoGNNBlock
nn = GeoGNNBlock().cuda()
out = nn.forward(graph, node_feats, edge_feats)
out.shape

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


torch.Size([10, 32])

In [17]:
src, dst = graph.edges()
mask = src < dst
src


(tensor([0, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 7, 8, 8, 9],
        device='cuda:0', dtype=torch.int32),
 tensor([1, 0, 2, 1, 3, 7, 2, 4, 3, 5, 4, 6, 5, 7, 2, 6, 8, 7, 9, 8],
        device='cuda:0', dtype=torch.int32))

In [122]:
import numpy as np, torch
# new_mol = Chem.AddHs(mol)
# res = AllChem.EmbedMultipleConfs(new_mol, numConfs=10)
# ### MMFF generates multiple conformations
# res = AllChem.MMFFOptimizeMoleculeConfs(new_mol)
# new_mol = Chem.RemoveHs(new_mol)
# index = np.argmin([x[1] for x in res])
# conf = new_mol.GetConformer(id=int(index))

atom_pos = torch.from_numpy(conf.GetPositions())[:mol.GetNumAtoms()]
src_node_idx, dst_node_idx = [tensor.long() for tensor in g.edges()]
# print(atom_pos[g.edges()[0].cpu()])
# g.edges()[0]

atom_pos[src_node_idx].dtype
# torch.from_numpy(atom_pos[dst_node_idx] - atom_pos[src_node_idx])

torch.float64

In [116]:
type(atom_pos)

numpy.ndarray

In [4]:
import numpy as np
import torch
centers = torch.tensor([1,2,3])
centers = torch.reshape(centers.float(), [1, -1])
x = torch.tensor([4,5,6,7,8])
x = torch.reshape(x, [-1, 1])
print(x.shape)
torch.exp(-10.0 * torch.square(x - centers)).shape

torch.Size([5, 1])


torch.Size([5, 3])

In [3]:
a,b = zip(*({'a':1, 'b':2}).items())
a,b

(('a', 'b'), (1, 2))

In [6]:
import dgl, torch, numpy as np
from rdkit import Chem
from rdkit.Chem import rdchem, AllChem
from dgl import DGLGraph
from typing import TypeAlias, Any, Callable, Literal
from dataclasses import dataclass


Atom: TypeAlias = rdchem.Atom | Any
Bond: TypeAlias = rdchem.Bond
Mol: TypeAlias = rdchem.Mol
Conformer: TypeAlias = rdchem.Conformer

class Utils:
    RdChemEnum: TypeAlias = Any     # RdChem's enums have no proper typings.
    @staticmethod
    def _rdchem_enum_to_list(rdchem_enum: RdChemEnum) -> list[RdChemEnum]:
        """Converts an enum from `rdkit.Chem.rdchem` (eg. `rdchem.ChiralType`)
        to a list of all the possible enum valuess.

        Args:
            rdchem_enum (RdChemEnum): An enum defined in `rdkit.Chem`.

        Returns:
            list[RdChemEnum]: All possible enum values in a list.
        """
        return [rdchem_enum.values[i] for i in range(len(rdchem_enum.values))]


    FeatureCategory: TypeAlias = Literal['atom_feats', 'bond_feats']
    FeatureName: TypeAlias = str
    GetValueFn: TypeAlias = Callable[[Atom], Bond]

    @dataclass
    class Feature:
        get_value: Callable[[Atom | Bond], Any]
        """Gets the feature value from an `rdchem.Atom` / `rdchem.Bond` instance."""
        possible_values: list[Any]
        """All possible values this feature can take on."""

    FEATURES: dict[FeatureCategory, dict[FeatureName, Feature]] = {
        'atom_feats': {
            'atomic_num': Feature(
                lambda atom: atom.GetAtomicNum(),
                list(range(1, 119)) + ['misc'],
            ),
            'chiral_tag': Feature(
                lambda atom: atom.GetChiralTag(),
                _rdchem_enum_to_list(rdchem.ChiralType),
            ),
            'degree': Feature(
                lambda atom: atom.GetDegree(),
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'misc'],
            ),
            'formal_charge': Feature(
                lambda atom: atom.GetFormalCharge(),
                [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'misc'],
            ),
            'hybridization': Feature(
                lambda atom: atom.GetHybridization(),
                _rdchem_enum_to_list(rdchem.HybridizationType),
            ),
            'is_aromatic': Feature(
                lambda atom: int(atom.GetIsAromatic()),
                [0, 1],
            ),
            'total_numHs': Feature(
                lambda atom: atom.GetTotalNumHs(),
                [0, 1, 2, 3, 4, 5, 6, 7, 8, 'misc'],
            ),
        },
        'bond_feats': {
            'bond_dir': Feature(
                lambda bond: bond.GetBondDir(),
                _rdchem_enum_to_list(rdchem.BondDir),
            ),
            'bond_type': Feature(
                lambda bond: bond.GetBondType(),
                _rdchem_enum_to_list(rdchem.BondType),
            ),
            'is_in_ring': Feature(
                lambda bond: int(bond.IsInRing()),
                [0, 1],
            ),
        }
    }

    @staticmethod
    def smiles_to_graph(smiles: str) -> DGLGraph:
        """Convert a molecule's SMILES string into a DGL graph.

        Args:
            smiles (str): A molecule's SMILES string.

        Returns:
            DGLGraph: The molecule in graph form.
        """
        mol = AllChem.MolFromSmiles(smiles)
        # mol, conf = Utils._generate_conformer(mol)

        # Create an undirected DGL graph with all the molecule's nodes and edges.
        num_bonds = mol.GetNumBonds()
        edges = torch.zeros(num_bonds, dtype=torch.int32), torch.zeros(num_bonds, dtype=torch.int32)
        for i, bond in enumerate(mol.GetBonds()):
            edges[0][i] = bond.GetBeginAtomIdx()
            edges[1][i] = bond.GetEndAtomIdx()
        graph = dgl.graph(edges, idtype=torch.int32)

        # Add node features.
        for feat_name, feat in Utils.FEATURES['atom_feats'].items():
            graph.ndata[feat_name] = torch.tensor([feat.get_value(atom) for atom in mol.GetAtoms()])

        # Add edge features.
        for feat_name, feat in Utils.FEATURES['bond_feats'].items():
            graph.edata[feat_name] = torch.tensor([feat.get_value(bond) for bond in mol.GetBonds()])

        graph = dgl.to_bidirected(graph, copy_ndata=True)   # Convert to undirected graph.
        # graph = graph.to('cuda:0')  # Copies graph to GPU. (https://docs.dgl.ai/guide/graph-gpu.html)
        return graph

    @staticmethod
    def _generate_conformer(mol: Mol, numConfs: int = 10) -> tuple[Mol, Conformer]:
        new_mol = Chem.AddHs(mol)
        res = AllChem.EmbedMultipleConfs(new_mol, numConfs=numConfs)
        ### MMFF generates multiple conformations
        res = AllChem.MMFFOptimizeMoleculeConfs(new_mol)
        new_mol = Chem.RemoveHs(new_mol)
        index = np.argmin([x[1] for x in res])
        conf = new_mol.GetConformer(id=int(index))
        return new_mol, conf

In [25]:
bonds_tensor = torch.zeros(2, mol.GetNumBonds(), dtype=torch.long)
for i, bond in enumerate(mol.GetBonds()):
    begin_atom_idx = bond.GetBeginAtomIdx()
    end_atom_idx = bond.GetEndAtomIdx()
    bonds_tensor[0, i] = begin_atom_idx
    bonds_tensor[1, i] = end_atom_idx

In [32]:
edges

In [14]:
[atom for atom in mol.GetAtoms()]